## Reinforcement Learning for SMS Messaging to Improve Medication Adherence - Roybal

In [1]:
import sys
import time
import dateutil
import dateutil.parser
from azure.cognitiveservices.personalizer import PersonalizerClient
from azure.cognitiveservices.personalizer.models import RankRequest
from msrest.authentication import CognitiveServicesCredentials
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime
from collections import Counter
import string
import pickle
import json
import pytz
import os
import re

In [2]:
from patient_data import import_pt_data,export_pt_data, new_empty_pt_data
from pillsy_parser import import_Pillsy, find_rewards
from driverReward import get_reward_update,send_rewards
from redcap_parser import import_redcap, update_pt_data_with_redcap
from driverRank import run_ranking, write_sms_history
from control_disconnection import check_control_disconnectedness

### Start Program Timer

In [3]:
# run_time = datetime.now()
# testing_flag = input("Testing with another date? y/n: ")
# if testing_flag.lower() == "y":
#     print("Set testing run time: ")
#     run_time_yyyy_mm_dd_input = input("Enter the testing date: YYYY-MM-DD ")
#     timestamp = "10:30 AM " + run_time_yyyy_mm_dd_input
#     run_time = dateutil.parser.parse(timestamp)
# run_time = pytz.timezone("America/New_York").localize(run_time)
# run_time

run_time = pytz.timezone("America/New_York").localize(dateutil.parser.parse("10:30 AM 2020-12-14"))
run_time 

datetime.datetime(2020, 12, 14, 10, 30, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)

### Set Up MS Azure Personalizer Client

Defining and Instantiating a Personalizer Client:

Personalizer Keys:
* In the Microsoft Azure Dashboard, navigate to our bwh-pharmacoepi-roybal-dev-use2-cog Cognitive Services page.
* Within the Keys and Endpoint section, copy either Key 1 or Key 2 to enter as the Personalizer Key.

Personalizer Endpoint:
* https://bwh-pharmacoepi-roybal-dev-use2-cog.cognitiveservices.azure.com/

In [4]:
with open(os.path.join("..", ".keys", "azure-personalizer-key.txt"), 'r') as f:
     personalizer_key = f.read().rstrip()
personalizer_endpoint = "https://bwh-pharmacoepi-roybal-dev-use2-cog.cognitiveservices.azure.com/"
client = PersonalizerClient(personalizer_endpoint, CognitiveServicesCredentials(personalizer_key))

## Reward Step

If we've already initiated the trial, we will have:
* Pre-existing patient dataset in need of reward updates
* Pillsy data from yesterday to determine reward
If this is study initiation, this step will just load an empty patient dictionary and null pillsy dataset.

In [5]:
pt_data = import_pt_data(run_time)
new_pillsy_data = import_Pillsy(run_time)

if not pt_data.empty and new_pillsy_data is not None:

    # From Pillsy data, computes the Rewards to send to Personalizer for each patient's Rank calls from yesterday's run.
    pt_data = find_rewards(new_pillsy_data, pt_data, run_time)

    # using updated patient data (new pillsy + patient data), format the rewards to Personalizer into a dataframe
    rewards_to_send = get_reward_update(pt_data, run_time)

    # actual call to personalizer
    send_rewards(rewards_to_send, client)
    export_pt_data(pt_data, run_time, "reward")

2020-12-03 17:45:00-05:00
record_id: 13
Computing... reward_value_t0 in pillsy_yesterday_subset with # med: 3
OPEN taken_event: 2020-12-13 09:15:00-05:00
drug_freq != taken => second_pass starting...
drug: Medication 1 BID
num taken: 1
drug_freq: 2
drug_adherence 0.5
OPEN taken_event: 2020-12-13 22:54:00-05:00
drug_freq != taken => second_pass starting...
drug: Medication 2 BID
num taken: 1
drug_freq: 2
drug_adherence 0.5
sum_timeframe_adherence: 1.0
num_pillsy_meds: 3
taken_over_expected: 0.3333333333333333
reward_value_t0: 0.3333333333333333
Computing... reward_value_t1 in pillsy_two_day_ago_subset with # med: 3
OPEN taken_event: 2020-12-12 09:31:00-05:00
drug_freq != taken => second_pass starting...
diff_second > two_hr_45_min -> taken:  2020-12-12 21:41:00-05:00
drug: Medication 1 BID
num taken: 2
drug_freq: 2
drug_adherence 1.0
OPEN taken_event: 2020-12-12 04:46:00-05:00
drug_freq != taken => second_pass starting...
diff_second > two_hr_45_min -> taken:  2020-12-12 19:23:00-05:00


reward_val:  0.0
event_id:  13_9_frame
reward_val:  0.0
event_id:  nan
reward_val:  0.0
event_id:  13_9_social
reward_val:  0.0
event_id:  13_9_content
reward_val:  0.0
event_id:  13_9_reflective
reward_val:  0.75
event_id:  15_10_frame
reward_val:  0.75
event_id:  15_10_history
reward_val:  0.75
event_id:  15_10_social
reward_val:  0.75
event_id:  15_10_content
reward_val:  0.75
event_id:  15_10_reflective
reward_val:  0.0
event_id:  15_9_frame
reward_val:  0.0
event_id:  nan
reward_val:  0.0
event_id:  15_9_social
reward_val:  0.0
event_id:  15_9_content
reward_val:  0.0
event_id:  15_9_reflective
reward_val:  0.0
event_id:  16_9_frame
reward_val:  0.0
event_id:  nan
reward_val:  0.0
event_id:  16_9_social
reward_val:  0.0
event_id:  16_9_content
reward_val:  0.0
event_id:  16_9_reflective
reward_val:  0.0
event_id:  17_9_frame
reward_val:  0.0
event_id:  nan
reward_val:  0.0
event_id:  17_9_social
reward_val:  0.0
event_id:  17_9_content
reward_val:  0.0
event_id:  17_9_reflective
r

## Import/Update Patients

In [6]:
redcap_data = import_redcap(run_time)
pt_data = update_pt_data_with_redcap(redcap_data, pt_data, run_time)

## Rank Step
Call Personalizer to rank action features to find the correct text message to send today.

In [7]:
ranked_pt_data = new_empty_pt_data()
for index, patient in pt_data.iterrows():
    if patient["censor"] != 1 and patient["censor_date"] > run_time.date():
        patient = run_ranking(patient, client, run_time)
        ranked_pt_data = ranked_pt_data.append(patient)
export_pt_data(ranked_pt_data, run_time, "rank") # log

records_id:  13  rankresult:  0 0 0 0 1
records_id:  15  rankresult:  1 1 0 1 1
records_id:  16  rankresult:  1 0 0 0 1
records_id:  17  rankresult:  1 0 0 0 1
records_id:  21  rankresult:  1 0 0 0 1
records_id:  22  rankresult:  1 1 0 0 1
records_id:  26  rankresult:  1 1 0 0 1


## Output SMS and Patient Data

In [8]:
write_sms_history(ranked_pt_data, run_time)
export_pt_data(ranked_pt_data, run_time, "final") # input for tomorrow
check_control_disconnectedness(run_time) # check whether controls have connection problems

2020-12-03 17:45:00-05:00
OPEN taken_event: 2020-12-13 09:15:00-05:00
drug_freq != taken => second_pass starting...
drug: Medication 1 BID
num taken: 1
drug_freq: 2
drug_adherence 0.5
OPEN taken_event: 2020-12-13 22:54:00-05:00
drug_freq != taken => second_pass starting...
drug: Medication 2 BID
num taken: 1
drug_freq: 2
drug_adherence 0.5
OPEN taken_event: 2020-12-12 09:31:00-05:00
drug_freq != taken => second_pass starting...
diff_second > two_hr_45_min -> taken:  2020-12-12 21:41:00-05:00
drug: Medication 1 BID
num taken: 2
drug_freq: 2
drug_adherence 1.0
OPEN taken_event: 2020-12-12 04:46:00-05:00
drug_freq != taken => second_pass starting...
diff_second > two_hr_45_min -> taken:  2020-12-12 19:23:00-05:00
drug: Medication 2 BID
num taken: 2
drug_freq: 2
drug_adherence 1.0
OPEN taken_event: 2020-12-12 08:08:00-05:00
drug_freq != taken => second_pass starting...
diff_second > two_hr_45_min -> taken:  2020-12-12 20:08:00-05:00
drug: Medication 3 BID
num taken: 2
drug_freq: 2
drug_adh

To Do List:
  * Executability by RA / User friendliness
    * [x] - Lily done by using Jupyter notebook - Making this more user friendly than a  Command Line
    * [ ] - JOE TODO / help brainstorm - make jupyter notebook pretty and write a how to document for new non CS person to be able to execute this from Jupyter notebook
    * [ ] - JOE TODO / help brainstorm - Making this more user friendly than a Jupyter Notebook - to do by doing a main.py executable bash script
    * [ ] - JOE TODO / help brainstorm - Hooks into Pillsy/RedCap for data retrieval - need check in with constance for pillsy and ellen for redcap
    * [ ] - JOE TODO / help brainstorm - (probably not feasible) Hooks into SMS Platform to automate text sending - need check in with constance
    * [ ] - JOE TODO / help brainstorm - how to handle the definite 1 human entered variable of personalizer key - file to direct to or? - maybe as marco/elad
    * [ ] - Lily/JOE TODO / help - try to break this code in any way possible
    * [ ] - Lily/JOE TODO / help - debug, unit testing
    * [ ] - Lily/JOE TODO / help - run it fully several times based in dropbox together
    * [ ] - Lily/JOE TODO / help - Make a log file that will report high level information from running this like a run summary
        * elements to include:
            * start and end time of the run
            * how many patients were read in from each import statement
            * how many reward calls were successfull made
            * how many rank calls were successfull made
            * any other meta data that will help us debug and ensure this is all working as planned


